# Battery Data

## Introduction

Battery data can be recorded by many devices, including smartphones and watches. Battery information can provide insight about activity and sleep patterns. It is also necessary to interpret other data recorded by the device.

Battery data includes information about when the device is shut down, if applicable.

Battery data is expected to have the following columns (column names can be different, but in that case they must be provided as parameters):
- `user`: Subject ID
- `device`: Device ID
- `battery_level`: The charge level if the device as percentage of full charge.
- `battery_status`: Integer indicator of device status. Fully documented at https://awareframework.com/battery/.
    - Status `2`: charging
    - Status `3`: discharging
    - Status `5`: battery full
    - Status `4`: not charging
    - Status `1`: unknown
    - Status `-1`: shutdown
    - Status `-2`: rebooted


In this notebook, we will extract battery data from the Aware platform and infer users' behavioral patterns from their interaction with the phone. The below functions will be described in this notebook:

- `niimpy.preprocessing.battery.battery_shutdown_info`: returns the timestamp when the device is shutdown or rebooted
- `niimpy.preprocessing.battery.battery_occurrences`: returns the number of battery samples within a time range
- `niimpy.preprocessing.battery.battery_gaps`: returns the time gaps between two battery sample


## Read data

In [1]:
import niimpy
import niimpy.preprocessing.battery as battery
from niimpy import config
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = niimpy.read_csv(config.MULTIUSER_AWARE_BATTERY_PATH, tz='Europe/Helsinki')
data.head()

,user,device,time,battery_level,battery_status,battery_health,battery_adaptor,datetime
2020-01-09 02:20:02.924999952+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,74,3,2,0,2020-01-09 02:20:02.924999952+02:00
2020-01-09 02:21:30.405999899+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,73,3,2,0,2020-01-09 02:21:30.405999899+02:00
2020-01-09 02:24:12.805999994+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578529e+09,72,3,2,0,2020-01-09 02:24:12.805999994+02:00
2020-01-09 02:35:38.561000109+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,72,2,2,0,2020-01-09 02:35:38.561000109+02:00
2020-01-09 02:35:38.953000069+02:00,jd9INuQ5BBlW,3p83yASkOb_B,1.578530e+09,72,2,2,2,2020-01-09 02:35:38.953000069+02:00


## Feature extraction

By default, Niimpy data should be ordered by the timestamp in ascending order. We start by sorting the data to make sure it's compatible.

In [3]:
data = data.sort_index()

Next, we will use Niimpy to extract features from the data. These are useful for inspecting the data and can be part of a full analysis workflow.

Usin the `battery_occurrences` function, we can count the amount the battery samples every 10 minutes. This function requires the index to be sorted.

In [4]:
battery.battery_occurrences(data, {"resample_args": {"rule": "10min"}})

,user,occurrences,device
2019-08-05 14:00:00+03:00,dvWdLQesv21a,1,i8jmoIuoe12Mo
2019-08-05 14:30:00+03:00,dvWdLQesv21a,0,i8jmoIuoe12Mo
2019-08-05 15:00:00+03:00,dvWdLQesv21a,0,i8jmoIuoe12Mo
2019-08-05 15:30:00+03:00,dvWdLQesv21a,0,i8jmoIuoe12Mo
2019-08-05 16:00:00+03:00,dvWdLQesv21a,1,i8jmoIuoe12Mo
...,...,...,...
2020-01-09 21:30:00+02:00,jd9INuQ5BBlW,4,OWd1Uau8POix
2020-01-09 22:00:00+02:00,jd9INuQ5BBlW,2,OWd1Uau8POix
2020-01-09 22:30:00+02:00,jd9INuQ5BBlW,1,OWd1Uau8POix
2020-01-09 23:00:00+02:00,jd9INuQ5BBlW,3,OWd1Uau8POix


The above dataframe gives the battery information of all users. You can also get the information for an individual by passing a filtered dataframe.

In [5]:
f = niimpy.preprocessing.battery.battery_occurrences
data_filtered = data.query('user == "jd9INuQ5BBlW"')
individual_occurrences = battery.extract_features_battery(data_filtered, features={f: {"resample_args": {"rule": "10min"}}})
individual_occurrences.head()

<function battery_occurrences at 0x74700a921260> {'resample_args': {'rule': '10min'}}


,user,device,occurrences
2020-01-09 02:20:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,3
2020-01-09 02:30:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,5
2020-01-09 02:40:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,6
2020-01-09 02:50:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,6
2020-01-09 03:00:00+02:00,jd9INuQ5BBlW,3p83yASkOb_B,5


Next, you can extract the gaps between two consecutive battery samples with the `battery_gaps` function.

In [6]:
f = niimpy.preprocessing.battery.battery_gaps
gaps = battery.battery_gaps(data)
gaps

,user,device,battery_gap
2019-08-05 14:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0 days 00:00:00
2019-08-05 14:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 15:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 15:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 16:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0 days 02:18:38.523000002
...,...,...,...
2020-01-09 21:30:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:05:41.859499991
2020-01-09 22:00:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:14:10.238499999
2020-01-09 22:30:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:21:09.899999857
2020-01-09 23:00:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:13:20.001333395


Knowing when the phone is shutdown is essential if we want to infer the usage behaviour of the subjects. This can be done by calling the `shutdown_info` function. The function returns the timestamp when the phone is shut down or rebooted (e.g: battery_status = -1).

In [7]:
shutdown = battery.shutdown_info(data, battery_column_name = 'battery_status')
shutdown

,user,device,time,battery_level,battery_status,battery_health,battery_adaptor,datetime
2019-08-07 10:37:11.308000088+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,1.565163e+09,2,-1,2,0,2019-08-07 10:37:11.308000088+03:00
2019-08-07 10:37:11.308000088+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565163e+09,2,-1,2,0,2019-08-07 10:37:11.308000088+03:00
2019-08-07 10:37:11.322999954+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,1.565163e+09,2,-1,2,0,2019-08-07 10:37:11.322999954+03:00
2019-08-07 10:37:11.322999954+03:00,iGyXetHE3S8u,Cq9vueHh3zVs,1.565163e+09,2,-1,2,0,2019-08-07 10:37:11.322999954+03:00


## Extracting features with the extract_features call

We have seen above how to extract battery features using `niimpy`. Sometimes, we need more than one features and it would be inconvenient to extract everything one by one. `niimpy` provides a `extract_feature` call to allow you extracting all the features available and combining them into a single data frame. The extractable features must start with the prefix `battery_`.

In [8]:
# Start by defining the feature name
f0 = niimpy.preprocessing.battery.battery_occurrences
f1 = niimpy.preprocessing.battery.battery_gaps
f2 = niimpy.preprocessing.battery.battery_charge_discharge

# The extract_feature function requires a features parameter. 
# This parameter accepts a dictionary where the key is the feature name and value
# is a dictionary containing values passed to the function.
features = battery.extract_features_battery(
    data,
    features={f0: {'rule': "10min"},
    f1: {},
    f2: {}
})
features.head()

<function battery_occurrences at 0x74700a921260> {'rule': '10min'}
<function battery_gaps at 0x74700a921300> {}
<function battery_charge_discharge at 0x74700a9213a0> {}


,user,device,occurrences,battery_gap,bdelta,charge/discharge
2019-08-05 14:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,1,0 days 00:00:00,0.0,NaN
2019-08-05 14:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0,NaT,NaN,NaN
2019-08-05 15:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0,NaT,NaN,NaN
2019-08-05 15:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0,NaT,NaN,NaN
2019-08-05 16:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,1,0 days 02:18:38.523000002,-1.0,-0.00012


In [9]:
f1(data)

,user,device,battery_gap
2019-08-05 14:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0 days 00:00:00
2019-08-05 14:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 15:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 15:30:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,NaT
2019-08-05 16:00:00+03:00,dvWdLQesv21a,i8jmoIuoe12Mo,0 days 02:18:38.523000002
...,...,...,...
2020-01-09 21:30:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:05:41.859499991
2020-01-09 22:00:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:14:10.238499999
2020-01-09 22:30:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:21:09.899999857
2020-01-09 23:00:00+02:00,jd9INuQ5BBlW,OWd1Uau8POix,0 days 00:13:20.001333395
